In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

df = pd.read_csv("qoute_dataset.csv")


In [3]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [4]:
df.shape

(3038, 2)

In [5]:


quotes = df['quote']
quotes.head()

0    “The world as we have created it is a process ...
1    “It is our choices, Harry, that show what we t...
2    “There are only two ways to live your life. On...
3    “The person, be it gentleman or lady, who has ...
4    “Imperfection is beauty, madness is genius and...
Name: quote, dtype: object

In [6]:
# CONVERTINGALLQUOTES TO LOWERCASE
quotes=quotes.str.lower()

In [7]:
import string
# HERE WE REMOVE ALL PUNCTUATIONS LIKE COMMAS ETC FROM QUOTES
translator= str.maketrans('','',string.punctuation) 
quotes= quotes.apply(lambda x: x.translate(translator))

In [8]:
quotes.head()

0    “the world as we have created it is a process ...
1    “it is our choices harry that show what we tru...
2    “there are only two ways to live your life one...
3    “the person be it gentleman or lady who has no...
4    “imperfection is beauty madness is genius and ...
Name: quote, dtype: object

In [9]:


from tensorflow.keras.preprocessing.text import Tokenizer

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [10]:
vocab_size = 10000
#  OOV TOKEN HANDLES UNKNOWN WORDS 
tokenizer  = Tokenizer(num_words=vocab_size)
# LEARNS A WORD DICTIONARY FROM ALL SENTENCES
tokenizer.fit_on_texts(quotes)

In [11]:
# CONVERTING RANDOM WORDS INTO WORDS INDEX THAT WILL SHOWS IN FORM OF INDEX
word_index = tokenizer.word_index
print(len(word_index))
# WE ARE HERE PRINTING IN LIST FORM UP TO INDEX 10
list(word_index.items())[:10]

8978


[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10)]

In [12]:
# IT CONVERTS SEQUENCE INTO NUMBERS SEQUENCES
# "I love this product" → [12, 45, 6, 98] 
sequences = tokenizer.texts_to_sequences(quotes)

In [13]:
# HERE WE PRINT FIRST 3 QUOTES IN FORM OF SEQUENCES
for i in range(3):
    print(quotes[i])

“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”
“it is our choices harry that show what we truly are far more than our abilities”
“there are only two ways to live your life one is as though nothing is a miracle the other is as though everything is a miracle”


In [14]:
# HERE WE PRINT FIRST 3 QUOTES IN FORM OF SEQUENCES(NUMBERS)
for i in range(3):
    print(sequences[i])

[713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12, 809, 104, 752, 70, 2461]
[947, 7, 70, 871, 373, 9, 433, 21, 19, 465, 14, 294, 52, 54, 70, 3676]
[1337, 14, 53, 201, 714, 3, 81, 15, 36, 37, 7, 29, 329, 93, 7, 5, 1157, 1, 101, 7, 29, 329, 126, 7, 5, 3677]


In [15]:
X= []
y= []
# X contains partial sequences

# y contains the next token to predict

for seq in sequences:
    # RANGE 1 SE START HORI HAI AUR LEN TAK JAYEGA
  for i in range(1,len(seq)):
    # INPUT ME SABSE PAHELE SE SARE AYEGE SEQUENCE 
    input_seq = seq[:i]
    # OUTPUT ME LAST JO SEQUENCE HOGA YA JO LAST WORD YA NUMBER HOGA WO AYEGA 
    output_seq = seq[i]
    #  HERE WE ARE APPENDING 
    X.append(input_seq)
    y.append(output_seq)

In [16]:
len(X)

85271

In [17]:
len(y)

85271

In [18]:
# FINDS LENGTH OF LONGEST SENTENCE 
# BASICALLY IT FINDS MAXM LENGTH THAT HOW MUCH LEN OF NUMBERS IS FORMING FROM WORDS TO NUMBERS SEQUENCE 
max_len = max(len(x) for x in X)
print(max_len)

745


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# HERE WE ARE MAKING SAME LENGTH BY ADDING 0 AT THE BEGGINING(PRE) OF SEQUENCES
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')


In [20]:

y = np.array(y)

In [21]:

X_padded.shape

(85271, 745)

In [22]:
#  HERE WE ARE DOING ONE HOT ENCODEING FOR CATEGORICAL OUTPUT

from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size)

In [23]:
y.shape

(85271,)

In [24]:
y_one_hot.shape

(85271, 10000)

In [ ]:
# Embedding	Convert word IDs → dense vectors
# SimpleRNN	Core RNN layer
# Dense	Final output layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense

In [26]:
embedding_dim = 50
rnn_units = 128

In [ ]:
#  HERE WE ARE MAKING A SEQUENTIAL MODEL

rnn_model = Sequential()


# 1. Embedding Layer: Converts word indices into dense vectors of a fixed size.
rnn_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
# 2. SimpleRNN Layer: Processes the sequence data.BASICALLY HERE WE USED SIMPLE RNN
rnn_model.add(SimpleRNN(units=rnn_units))
#  OUTPUT LAYER
# A Dense layer with softmax activation to predict the next word in the vocabulary.
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
rnn_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [35]:
rnn_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
# NOW HERE WE ARE PERFORMING WITH LSTM MODEL
lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:

lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [38]:

lstm_model.summary()
     

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#  HERE WE LOAD OUR MODEL 
from tensorflow.keras.models import load_model

lstm_model = load_model("lstm_model (1).h5")

In [42]:

lstm_model.save("lstm_model.h5")
     

In [ ]:
index_to_word = {}
#  HERE WE ADD A LOOP IN WHICH WE ACCEPTING WORD AND INDEX AND KEEPING AT WHAT INDEX WHICH WORD IS THERE
# AND ADD IT INTO THE INDEX_TO_WORD DICTIONARY
for word, index in word_index.items():
    index_to_word[index] = word

In [ ]:

# MAKE ALL SENTENCE INTO THE SAME LENGTH
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:

# HERE WE MAKE A FUNCTION FOR WORD PREDICTOR 
# WHICH ACCEPTS MODEL LIKE SIMPLERNN, LSTM ETC...
# TOKENZER , TEXT AND MAX_LENGTH
def predictor(model,tokenizer,text,max_len):
  #  CONVERTS TEXT INTO LOWER CASE 
  text = text.lower()
# CONVERT TEXT INTO TOKENIZE[TOKENS] FROM 0TH POSITION
  seq = tokenizer.texts_to_sequences([text])[0]
  # PAD_SEQUENCE MEANS ADD 0 IN EMPTY POSITION...HERE PADDING IS PRE SO 0 IS ADD ON STARTING EMPTY SPACES 
  seq = pad_sequences([seq], maxlen=max_len, padding='pre')

# PREDICTION
  pred = model.predict(seq,verbose = 0)
  #  CONVERT FOR NUMERICAL OPERATION FORM 
  pred_index = np.argmax(pred)
  # STORE IN THE DICTIONARY
  return index_to_word[pred_index]


In [ ]:
# PASSING SENTENCE TO PREDICT NEXT WORD
seed_text = "what are you"
# GIVING ALL PARAMS TO FUNCTION
next_word = predictor(lstm_model,tokenizer,seed_text,max_len)
print(next_word)

worrying


In [ ]:
#  TEXT GENERATOR FUNCTION 
# HERE WE HAVE ONE EXTRA THING WHICH IS N_WORDS FOR OUR TEXT GENERATION
def generate_text(model,tokenizer,seed_text,max_len,n_words):
  for _ in range(n_words):
    next_word = predictor(model,tokenizer,seed_text,max_len)
    # AGAR NEXT_WORD EMPTY HAI TO BREAK KRDO
    if next_word == "":
      break
    # NHI TO SEED_TEXT ME ADD KR DO
    seed_text += " " + next_word
  return seed_text

In [50]:
seed = "are you a "
generate_text = generate_text(lstm_model,tokenizer,seed,max_len,10)
print(generate_text)

are you a  thousand times i wrote the less if it does not


In [ ]:
# MAKING PICKLE FILE OF TOKENIZER
import pickle
with open("tokenizer.pkl", "wb") as f:
  pickle.dump(tokenizer, f)

In [ ]:
# MAKING PICKLE FILE OF MAX_LEN
with open("max_len.pkl", "wb") as f:
  pickle.dump(max_len, f)